# Converting Scraped Data into IDPCentral

Alasdair J G Gray ([ORCID:0000-0002-5711-4872](http://orcid.org/0000-0002-5711-4872)), _Heriot-Watt University, Edinburgh, UK_

Petros Papadopoulos ([ORCID:0000-0002-8110-7576](https://orcid.org/0000-0002-8110-7576)), _Heriot-Watt University, Edinburgh, UK_

Ivan Mičetić ([ORCID:0000-0003-1691-8425](https://orcid.org/0000-0003-1691-8425)), _University of Padua, Italy_

Andras Hatos ([ORCID:0000-0001-9224-9820](https://orcid.org/0000-0001-9224-9820)), _University of Padua, Italy_

## Introduction

IDPCentral is the idea of having a central registry of proteins that are known to be disordered.

We aim to populate the content of the registry with Bioschemas markup that has been scraped using the BMUSE tool.

This notebook goes through the steps of converting the scraped content into the IDPCentral data model.

### To Dos

- ~~Return provenance data to IDPcentral: straightforward add properties from graph~~
- ~~Consider changing base namespace for IDPKG graph to either one you own or Wikidata~~
  - ~~Is it valid for us to hang our properties off a UniProt ID~~
  - ~~If we are using UniProt IDs we need to be consistent in our usage~~
  - ~~Using UniProt accession with Bioschemas namespace~~
- Add metadata properties and statements
  - Do as a separate query
  - Encoutering problems creating the intermediary nodes, may be better to put KG in named graph and hang metadata off the graph, alhtough this would lose some of the advantages of mixing the data together in one graph
- Retrieve UniProt label for IDPcentral using SPARQLWrapper to make external call and add properties
- Do we want to connect to Wikidata IDs?
- ~~IDPcentral is not getting updated with entries from mobidb~~
- MobiDB data getting mangled by BMUSE (currently testing with manually fixed file)
- Invesitage if [rdf-config](https://github.com/dbcls/rdf-config/blob/master/doc/spec.md) can be used to document the generated KG model

## IDPKG Data Model

The IDPKG data model reuses ideas from [Wikidata](https://www.mediawiki.org/wiki/Wikibase/DataModel) whereby every statement loaded contains a provenance link as to where it was acquired.

- [ ] Document IDPCentral Model

### Identifiers

For the identifiers in the KG we are using the UniProt accession with Bioschemas namespace. This produces unique IRIs that are distinct from UniProts. While this means that there is a level of indirection in the integration, it relies on `sameAs` links, it provides the flexibility to choose whether to combine the data.

## Data Sources

The following databases have been scraped to populate IDPCentral
- [DisProt](https://www.disprot.org/)
- [MobiDb](https://mobidb.bio.unipd.it/)
- [Protein Ensemble Database](https://proteinensemble.org/)

## Conversion using RDFlib

This is an attempt to achieve the same functionality without using a triplestore.

Load in the RDFLib library.

In [ ]:
from rdflib import ConjunctiveGraph, Graph

Template library used to template queries.

In [ ]:
from string import Template

Import functions to list files in directory

In [ ]:
from glob import glob

Import the ability to create a UUID

In [ ]:
import uuid

Prepare query to extract UniProt and DisProt IRIs.

In [ ]:
idQuery = """
PREFIX schema: <https://schema.org/>
SELECT ?proteinIRI ?uniprot
WHERE {
    GRAPH ?g {
        ?proteinIRI a schema:Protein ;
            schema:sameAs ?uniprot .
        FILTER regex(str(?uniprot), "^(https://www|http://purl).uniprot.org/uniprot/")
    }
}
"""

Templated query for creating the direct properties for the entity, i.e. the data.

In [ ]:
createEntityQuery = Template("""
# Query to convert DisProt scraped data to IDPCentral model
# Defensive query: assumes that data does not conform to Protein profile

PREFIX bs: <https://bioschemas.org/entity/>
PREFIX pav: <http://purl.org/pav/>
PREFIX schema: <https://schema.org/>

CONSTRUCT {
    bs:${bsAccession} a schema:Protein ;
        schema:identifier ?identifier ;
        schema:name ?name ;
        schema:associatedDisease ?associatedDisease ;
        schema:description ?description ;
        schema:hasSequenceAnnotation ?annotation ;
        schema:isEncodedByBioChemEntity ?encodedBy ;
        schema:taxonomicRange ?taxonomicRange ;
        schema:url ?url ;
        schema:alternateName ?alternateName ;
        schema:bioChemInteraction ?bioChemInteraction ;
        schema:bioChemSimilarity ?bioChemSimilarity ;
        schema:hasBioChemEntityPart ?bioChemEntity ;
        schema:hasBioPloymerSequence ?sequence ;
        schema:hasMolecularFunction ?molFunction ;
        schema:hasRepresentation ?representation ;
        schema:image ?image ;
        schema:isInvolvedInBiologicalProcess ?process ;
        schema:isLocatedInSubcellularLocation ?cellularLocation ;
        schema:isPartOfBioChemEntity ?parentEntity ;
        schema:sameAs ?sameAs , ?s ;
        pav:retrievedFrom ?source.
}
WHERE {
    GRAPH ?g {
# Bioschemas Minimal Properties
        ?s a schema:Protein .
        OPTIONAL {?s schema:identifier ?identifier }
        OPTIONAL {?s schema:name ?name }
## Bioschemas Recommended properties
        OPTIONAL {?s schema:associatedDisease ?associatedDisease}
        OPTIONAL {?s schema:description ?description}
        OPTIONAL {?s schema:hasSequenceAnnotation ?annotation }
        OPTIONAL {?s schema:isEncodedByBioChemEntity ?encodedBy}
        OPTIONAL {?s schema:taxonomicRange ?taxonomicRange }
        OPTIONAL {?s schema:url ?url}
## Bioschemas Optional properties
        OPTIONAL {?s schema:alternateName ?alternateName}
        OPTIONAL {?s schema:bioChemInteraction ?bioChemInteraction}
        OPTIONAL {?s schema:bioChemSimilarity ?bioChemSimilarity}
        OPTIONAL {?s schema:hasBioChemEntityPart ?bioChemEntity}
        OPTIONAL {?s schema:hasBioPloymerSequence ?sequence}
        OPTIONAL {?s schema:hasMolecularFunction ?molFunction}
        OPTIONAL {?s schema:hasRepresentation ?representation }
        OPTIONAL {?s schema:image ?image}
        OPTIONAL {?s schema:isInvolvedInBiologicalProcess ?process}
        OPTIONAL {?s schema:isLocatedInSubcellularLocation ?cellularLocation}
        OPTIONAL {?s schema:isPartOfBioChemEntity ?parentEntity}
        OPTIONAL {?s schema:sameAs ?sameAs }
    }
    ?g pav:retrievedFrom ?source ;
            pav:retrievedOn ?date .
}
""")

Query to retrieve data about annotations

In [ ]:
retrieveAnnotationsQuery = """
PREFIX schema: <https://schema.org/>
CONSTRUCT {
  ?s a schema:SequenceAnnotation ;
        schema:additionalProperty ?addProp ;
        schema:citation ?citation ;
        schema:creationMethod ?method ;
        schema:description ?description ;
        schema:editor ?editor ;
        schema:isPartOfBioChemEntity ?bioChemEntity ;
        schema:sequenceLocation ?seqLoc .
}
WHERE {
  graph ?g {
    ?s a schema:SequenceAnnotation .
    OPTIONAL {?s schema:additionalProperty ?addProp }
    OPTIONAL {?s schema:citation ?citation }
    OPTIONAL {?s schema:creationMethod ?method }
    OPTIONAL {?s schema:description ?description }
    OPTIONAL {?s schema:editor ?editor }
    OPTIONAL {?s schema:isPartOfBioChemEntity ?bioChemEntity }
    OPTIONAL {?s schema:sequenceLocation ?seqLoc }
  }
}
"""

Retrieve triples about PropertyValues.

In [ ]:
retrievePropertyValueQuery = """
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?s a schema:PropertyValue ;
        schema:name ?name ;
        schema:value ?value .
}
where {
    graph ?g {
        ?s a schema:PropertyValue .
        OPTIONAL {?s schema:name ?name }
        OPTIONAL {?s schema:value ?value }
    }
}
"""

Retrieve triples about SequenceRange.

In [ ]:
retrieveSequenceRangeQuery = """
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?s a schema:SequenceRange ;
        schema:rangeStart ?start ;
        schema:rangeEnd ?end .
}
where {
    graph ?g {
        ?s a schema:SequenceRange .
        OPTIONAL {?s schema:rangeStart ?start }
        OPTIONAL {?s schema:rangeEnd ?end}
    }
}
"""

Templated query for creating the links to the provenance for each statement in the KG.

In [ ]:
createEntityProvenanceQuery = Template("""
PREFIX bs: <https://bioschemas.org/entity/>
PREFIX bsp: <https://bioschemas.org/ns/p/>
PREFIX bss: <https://bioschemas.org/ns/s/>
PREFIX bsr: <https://bioschemas.org/reference/>
PREFIX pav: <http://purl.org/pav/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>
CONSTRUCT {
#    bs:${bsAccession} a schema:Protein ;
#        schema:identifier ?identifier ;
#        schema:name ?name ;
#        schema:hasSequenceAnnotation ?annotation ;
#        schema:taxonomicRange ?taxonomicRange ;
#        schema:hasRepresentation ?representation ;
#        schema:sameAs ?sameAs , <${proteinIRI}>.
    bs:${bsAccession} bsp:type [
        prov:wasDerivedFrom <${refNodeIRI}> ;
        bss:type schema:Protein
    ] .
    ?refNode pav:retrievedFrom ?source ;
        pav:retrievedOn ?date .
}
WHERE {
    GRAPH ?g {
# Bioschemas Minimal Properties
        <${proteinIRI}> a schema:Protein .#;
#            schema:identifier ?identifier ;
#            schema:name ?name ;
# Bioschemas Recommended properties
#            schema:hasSequenceAnnotation ?annotation ;
#            schema:taxonomicRange ?taxonomicRange ;
# Bioschemas Optional properties
#            schema:sameAs ?sameAs .
#        OPTIONAL {<${proteinIRI}> schema:hasRepresentation ?representation }
        ?g pav:retrievedFrom ?source ;
            pav:retrievedOn ?date .
    }
}
""")

Method for creating the KG entity and its metadata using the templated queries above.

In [ ]:
def createKGEntity(g, protein, uniprot):
    kgEntity = Graph()
    # Extract UniProt accession to use as an identifier in the Bioschemas namespace
    accession = uniprot[uniprot.rindex('/')+1:]
    query = createEntityQuery.substitute(proteinIRI=protein,bsAccession=accession)
    kgEntity += g.query(query)
    kgEntity += g.query(retrieveAnnotationsQuery)
    kgEntity += g.query(retrievePropertyValueQuery)
    kgEntity += g.query(retrieveSequenceRangeQuery)
#    # Attempt to generate provenance statements as per Wikidata
#     u = uuid.uuid1()
#     refNode = "https://bioschemas.org/reference/" + accession + "-" + str(u)
#     query = createEntityProvenanceQuery.substitute(proteinIRI=protein,bsAccession=accession,refNodeIRI=refNode)
#     print(query)
#     kgEntity += g.query(query)
    return kgEntity

Function to extract just the triples that IDPCentral are using in their UI

In [ ]:
idpQuery = """
PREFIX idp: <https://example.com/ipd/>
PREFIX pav: <http://purl.org/pav/>
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?entry_url idp:name ?entry_name ;
        idp:identifier ?entry_id ;
        idp:sameAs ?uniprot_acc ;
        idp:sequence_range [
            idp:sequence_id ?sequenceID ;
            idp:start ?start ;
            idp:end ?end ;
            idp:range_annotation ?range_annotation
        ] ;
        idp:resource_name ?source ;
        idp:last_update ?date.
}
WHERE {
    GRAPH ?g {
        ?entry_url a schema:Protein ;
            schema:name ?entry_name ;
            schema:identifier ?entry_id ;
            schema:hasSequenceAnnotation ?sequenceID ;
            schema:sameAs ?uniprot_acc .
        FILTER regex(str(?uniprot_acc), "^(https://www|http://purl).uniprot.org/uniprot/")
        ?sequenceID schema:sequenceLocation ?sequenceLocation ;
                  schema:additionalProperty/schema:value/schema:name ?range_annotation .
        ?sequenceLocation schema:rangeStart ?start ;
            schema:rangeEnd ?end.
        ?g pav:retrievedFrom ?source ;
            pav:retrievedOn ?date .
    }
}
"""
def idpExtraction(g):
    return g.query(idpQuery)

Function to run over all files in a specified directory

In [ ]:
def processDataFiles(idpKG, idpModel, directoryLocation):
    processed = 0
    for file in glob(directoryLocation + "*.nq"):
        print("\tProcessing file: %s" % file)
        g = ConjunctiveGraph()
        g.parse(file, format="nquads")
        print("\tSource has %s statements." % len(g))
        # Extract statements for IDPCentral
        idpModel += idpExtraction(g)
        print("\tIDPcentral has %s statements." % len(idpModel))
        # Extract DisProt and UniProt IRIs
        results = g.query(idQuery)
#         print("\tID query result has %s statements." % len(results))
        # Convert to IDPCentral model
        for result in results:
#             print("\tProtein: %s\n\tUniProt: %s" % (result['proteinIRI'], result['uniprot']))
            resGraph = createKGEntity(g, result['proteinIRI'], result['uniprot'])
#             print("\tconvert query has %s statements." % len(resGraph))
            idpKG += resGraph
            print("\tIDPKG has %s statements." % len(idpKG))
        processed += 1
    return processed

Function to output data files for a graph

In [ ]:
def outputFiles(graph, label):
    # print(graph.serialize(format='nt'))
    print("%s has %s statements." % (label, len(graph)))
    graph.serialize(label+'.nt', format='nt')
    graph.serialize(label+'.jsonld', format='json-ld')
    print('Successfully written all triples to %s.nt' % label)

Read in each nq data file in turn

Process each file and convert into IDPCentral model

In [ ]:
idpKG = Graph()
idpModel = Graph()
totalProcessed = 0

print("Processing DisProt...")
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/disprot/")
print("Processed %d files" % numberOfFiles)
totalProcessed += numberOfFiles

print("Processing MobiDB...")
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/mobidb/")
print("Processed %d files" % numberOfFiles)
totalProcessed += numberOfFiles

print("Processing PED...")
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/ped/")
print("Processed %d files" % numberOfFiles)
totalProcessed += numberOfFiles

outputFiles(idpModel, "IDPCentral")
outputFiles(idpKG, "IDPKG")
print('Processed %d files' % totalProcessed)

assert (totalProcessed == 8), "Expected 8 data files to be present!"
assert (len(idpKG) == 204), "Expected 204 statements in IDP KG!"
assert (len(idpModel) == 58), "Expected 58 statements in IDPcentral Model!"
print('File processing finished successfully!')

Expecting:
- 8 files to have been procesed
  - 3 from DisProt
  - 2 from MobiDB
  - 3 from PED
- IDPKG should have 116 statements
  - DisProt
    - 53 after 2241.nq
    - 91 after 2243.nq
    - no additional statements from 2244.nq
  - mobiDB
    - 116 after 4283.nq
    - no additional statements from 5729.nq
  - IDPKG to have 204 statements
    - 134 statements after 6000.nq
    - 169 statements after 6001.nq
    - 204 statements after 5999.nq
- IDPCentral should have 58 statements

### Incorporating PED into the Pipeline

File 6000.nq corresponds to entry [PED00148](https://proteinensemble.org/PED00148). There is valid markup on the page. Something in BMUSE is preventing this from coming through to the scraped data.

File 5999.nq corresponds to entry [PED00001](http://proteinensemble.org/PED00001). The markup shows one protein component.

File 6001.nq corresponds to entry [PED00174](http://proteinensemble.org/PED00174). The markup shows 3 protein components.

- [X] Follow up with Petros as to whether BMUSE fixes mean that a new scrape of this content would now work.
- [ ] Rescraped data uses same IRI for DataRecord and BioChemEntity; BioChemEntity needs `#DR` removed from the `@id` value
- [ ] `createdWith` needs to be a IRI, it can have values from it
- [ ] Use of `#DR` has undesirable effect for BMUSE's use of `retrieveFrom`

## Querying the IDPKG

In [ ]:
def displayResults(results):
    for row in results.bindings:
        for col in row:
            print(col, row[col], end = '\t')
        print()

In [ ]:
# print("json", result.serialize(format="json"))
# for row in result:
#     print(row)
# print(result.serialize(format="json"))

### Find proteins in multiple datasets

Provenance information about the sources of triples has been included as a direct assertion on the protein resource. It will include multiple declarations if the triples about the protein have come from multiple pages. We cannot distinguish where any individual statement has come from. 

To find proteins with multiple sources, we need to group by the protein id and then use a `HAVING` clause to if there are more than two datasets. The datasets can be listed using a `GROUP_CONCAT` oeprator.

In [ ]:
proteinQuery = """
PREFIX schema: <https://schema.org/>
PREFIX pav: <http://purl.org/pav/>
SELECT ?protein (COUNT(?source) as ?numSources) (GROUP_CONCAT(?source;SEPARATOR=",") AS ?sources)
WHERE {
    ?protein a schema:Protein ;
        pav:retrievedFrom ?source .
}
GROUP BY ?protein
HAVING (COUNT(*) > 1)
"""
proteins = idpKG.query(proteinQuery)
displayResults(proteins)

In [ ]:
for protein in proteins:
    print(protein)

### Find proteins with annotations in multiple datasets
Again we exploit the multiplicity of identifiers to check for multiple datasets. However, we now explicitly check that there are two; again we could add more.

Note that we have changed to a `CONSTRUCT` query since we end up with a duplicate rows in a tuple bindings approach since the identifiers can be bound first one way and then the other.

#### Problem
The problem with this query is that it only checks that the protein appears in both datasets, it does not check that the annotations come from different datasets.

#### Possible Solution
For each protein and annotation, add a statement stating the sources that it has come from in the data, or alternatively have a named graph per source.

In [ ]:
proteinAnnotationQuery = """
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?protein a schema:Protein ;
        schema:name ?proteinName ;
        schema:identifier ?id1, ?id2 ;
        schema:hasSequenceAnnotation [
            schema:description ?annotationDescription 
        ].
}
WHERE {
    ?protein a schema:Protein ;
        schema:name ?proteinName ;
        schema:identifier ?id1, ?id2 ;
        schema:hasSequenceAnnotation ?annotation .
    OPTIONAL {?annotation schema:description ?annotationDescription }
    FILTER (?id1 != ?id2) .
}
    
"""
print(idpKG.query(proteinAnnotationQuery).serialize(format='n3'))

### Find proteins  with annotations in only one source

### Find proteins with annotations of type X

### Find annotations with identical ranges

### Find annotations with overlapping ranges